\Experimento 2 - KNN Imputer, 3sigma outlier detection, Z-Score normalization, MLP classifier

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
!pip install --quiet ucimlrepo
from ucimlrepo import fetch_ucirepo

In [2]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [3]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [4]:
import random

#Introducing Missing values (5%)
x = df.copy()

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      x[i][jndex] = np.NaN

df = x

<ipython-input-4-fbb7389abead>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN


In [5]:
df.isna().sum()

Area               704
Perimeter          663
MajorAxisLength    687
MinorAxisLength    670
AspectRatio        675
Eccentricity       642
ConvexArea         676
EquivDiameter      651
Extent             657
Solidity           676
Roundness          666
Compactness        674
ShapeFactor1       661
ShapeFactor2       671
ShapeFactor3       684
ShapeFactor4       692
dtype: int64

In [6]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,NaN,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,NaN,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,NaN,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,NaN,0.676884,0.996767
13609,42147.0,763.779,NaN,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [7]:
#Imputing Missing Values with KNN Imputer
from sklearn.impute import KNNImputer as knni

imputer = knni(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
df_imputed

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,163.949378,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.782621,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,41835.0,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001948,0.676884,0.996767
13609,42147.0,763.779,266.696861,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [8]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

          Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count     count           count           count       count   
Class                                                                  
BARBUNYA  1322      1322            1322            1322        1322   
BOMBAY     522       522             522             522         522   
CALI      1630      1630            1630            1630        1630   
DERMASON  3546      3546            3546            3546        3546   
HOROZ     1928      1928            1928            1928        1928   
SEKER     2027      2027            2027            2027        2027   
SIRA      2636      2636            2636            2636        2636   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                                      
BARBUNYA         1322       1322          1322   13

In [9]:
#Outlier Removal with 3sigma
from scipy import stats


print(f'Pre Outlier Shape: {df_imputed.shape}')
df_no_outliers = df_imputed.copy()

print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      upper = class_unique[feature].mean() + (3 * class_unique[feature].std())
      lower = class_unique[feature].mean() - (3 * class_unique[feature].std())
      excluded = pd.Series(class_unique[class_unique[feature] < lower].index)
      #print(excluded.values)
      df_no_outliers.drop(excluded.values, inplace = True, errors='ignore')

print(df_no_outliers.groupby('Class').count())
print(f'Pos Outlier Shape: {df_no_outliers.shape}')


Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
       Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRatio  \
Class                                                                   
0      1259       1259             1259             1259         1259   
1       458        458              458              458          458   
2      1578       1578             1578             1578         1578   
3      3411       3411             3411             3411         3411   
4      1802       1802             1802             1802         1802   
5      1920       1920             1920             1920         1920   
6      2536       2536             2536             2536         2536   

       Eccentricity  ConvexArea  EquivDiameter  Extent  Solidity  Roundness  \
Class                                                                         
0              1259        1259           1259    1259      1259       1259   
1               458         458            458     458    

In [10]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1259  1259      1259            1259            1259        1259   
BOMBAY     458   458       458             458             458         458   
CALI      1578  1578      1578            1578            1578        1578   
DERMASON  3411  3411      3411            3411            3411        3411   
HOROZ     1802  1802      1802            1802            1802        1802   
SEKER     1920  1920      1920            1920            1920        1920   
SIRA      2536  2536      2536            2536            2536        2536   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [11]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
2,30279.0,634.927,212.560556,168.756317,1.171067,0.520401,30600.0,196.347702,0.775688,0.989510,0.943852,0.923726,0.007020,0.002460,0.725734,0.999236,SEKER
3,30477.0,670.033,211.050155,184.039050,1.146768,0.489478,30970.0,196.988633,0.762402,0.984081,0.853080,0.933374,0.006925,0.003242,0.871186,0.999049,SEKER
4,30519.0,629.727,239.342008,182.737204,1.165591,0.513760,30847.0,197.124320,0.770682,0.989367,0.967109,0.925480,0.006979,0.003158,0.856514,0.998345,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12959,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.782621,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
12960,41835.0,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
12961,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001948,0.676884,0.996767,DERMASON
12962,42147.0,763.779,266.696861,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [12]:
#Normalization with Z-Score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,-0.853230,-1.153929,-1.320210,-0.638937,-1.583076,-2.244409,-0.853445,-1.078606,0.284987,0.357202,1.446274,1.864859,0.690687,2.421755,1.957524,0.934892,SEKER
1,-0.818679,-1.087893,-1.264849,-0.592059,-1.531755,-2.100692,-0.819751,-1.021978,0.580552,0.546330,1.267442,1.789410,0.611054,2.252973,1.871793,1.029850,SEKER
2,-0.787144,-1.036202,-1.268012,-0.756739,-1.690154,-2.571694,-0.788304,-0.971117,0.530049,0.533152,1.197199,2.035529,0.407552,1.255971,0.843890,1.076919,SEKER
3,-0.780199,-0.868443,-1.286002,-0.405962,-1.789747,-2.915796,-0.775518,-0.960017,0.253307,-0.927727,-0.397755,2.194338,0.321036,2.582350,2.336959,1.025022,SEKER
4,-0.778726,-1.061051,-0.949027,-0.435843,-1.712600,-2.645595,-0.779768,-0.957667,0.425770,0.494687,1.605856,2.064409,0.370358,2.440397,2.186348,0.829646,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12959,-0.372601,-0.439977,-0.360883,-0.362223,-0.125825,0.150181,-0.376795,-0.362059,0.674435,0.754211,0.718413,0.029585,0.260657,0.051333,-0.005507,0.840935,DERMASON
12960,-0.381791,-0.450475,-0.445988,-0.252775,-0.438513,-0.175863,-0.377279,-0.361869,1.035223,0.867348,0.813511,0.365175,0.105774,0.283156,0.334378,0.794712,DERMASON
12961,-0.371128,-0.441769,-0.446422,-0.241877,-0.454325,-0.194084,-0.374687,-0.360059,-0.422959,0.637824,0.750409,0.373035,0.099505,0.387802,0.342441,0.392018,DERMASON
12962,-0.370847,-0.420466,-0.623213,-0.262815,-0.385694,-0.116301,-0.371301,-0.359679,-0.934148,0.076406,0.565603,0.286249,0.138099,0.225837,0.253675,-0.036606,DERMASON


In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate


x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
classifier = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5, hidden_layer_sizes=(12, 3), random_state=1, verbose=True, learning_rate_init=0.3, tol=1e-3, max_iter=500)
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}


y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)
print('Experimento 4')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')


Iteration 1, loss = 1.24684022
Iteration 2, loss = 0.72337572
Iteration 3, loss = 0.46974261
Iteration 4, loss = 0.43165116
Iteration 5, loss = 0.38935719
Iteration 6, loss = 0.28419112
Iteration 7, loss = 0.25102877
Iteration 8, loss = 0.22155552
Iteration 9, loss = 0.22176371
Iteration 10, loss = 0.20836452
Iteration 11, loss = 0.20665156
Iteration 12, loss = 0.20951994
Iteration 13, loss = 0.19945365
Iteration 14, loss = 0.19369966
Iteration 15, loss = 0.19515406
Iteration 16, loss = 0.19887260
Iteration 17, loss = 0.20224151
Iteration 18, loss = 0.19607837
Iteration 19, loss = 0.18270802
Iteration 20, loss = 0.19995389
Iteration 21, loss = 0.20008260
Iteration 22, loss = 0.19308440
Iteration 23, loss = 0.19176581
Iteration 24, loss = 0.18789250
Iteration 25, loss = 0.19750689
Iteration 26, loss = 0.18274887
Iteration 27, loss = 0.18642734
Iteration 28, loss = 0.19463606
Iteration 29, loss = 0.18344305
Iteration 30, loss = 0.18485045
Training loss did not improve more than tol=0.001